In [ ]:
import pandas as pd
import numpy as np
import tifffile
from skimage.measure import regionprops, label
from skimage import measure
from tqdm import tqdm

# --- Load Data ---
mem_nuc_df = pd.read_csv('mem_nuc_overlap_table.csv')
mem_nuc_df['t'] = mem_nuc_df['t'] - 1

spots_df = pd.read_csv('full_csv_simple.csv')

# --- Merge DataFrames ---
merged_df_full = spots_df.merge(
    mem_nuc_df[['nuc_label', 'mem_label', 't']],
    left_on=['nuc_label', 'Spot frame'],
    right_on=['nuc_label', 't'],
    how='left'
)
merged_df_full = merged_df_full[~(merged_df_full.duplicated(subset=['Spot frame', 'nuc_label'], keep=False))]
merged_df_full_cropped = merged_df_full[~(merged_df_full.duplicated(subset=['Spot frame', 'mem_label'], keep=False))]

# --- Helper Functions ---
def get_largest_xy_area_and_bounding_box(segmented_image):
    """Find the largest bounding box side among all objects for cropping."""
    max_side = 0
    for prop in regionprops(segmented_image):
        x_axis = prop.bbox[5] - prop.bbox[2]
        y_axis = prop.bbox[4] - prop.bbox[1]
        longer_axis = max([x_axis, y_axis])
        if longer_axis > max_side:
            max_side = longer_axis
    return max_side + 5

def measure_objects(segmented_image):
    """Map label to centroid for all objects."""
    return {prop.label: prop.centroid for prop in regionprops(segmented_image)}

def process_label(segmented_image, membrane_labels, bbox, label_centroid_map):
    """Extract 2D binary crops centered on each label's centroid."""
    combined_2d_images = []
    for label in tqdm(membrane_labels):
        if np.isnan(label):
            combined_2d_images.append(np.zeros((150, 150)))
        else:
            label = int(label)
            centroid = label_centroid_map[label]
            z_slice = int(round(centroid[0]))
            min_row = int(round(centroid[1])) - bbox
            max_row = int(round(centroid[1])) + bbox
            min_col = int(round(centroid[2])) - bbox
            max_col = int(round(centroid[2])) + bbox
            cropped_2d = segmented_image[z_slice, min_row:max_row, min_col:max_col]
            cropped_2d = np.where(cropped_2d == label, cropped_2d, 0)
            binary_cropped_2d = np.where(cropped_2d > 0, 1, 0)
            combined_2d_images.append(binary_cropped_2d)
    return np.stack(combined_2d_images, axis=0)

def get_shape_descriptors(binary_image):
    """Extract shape descriptors from a binary image."""
    labeled_image = label(binary_image)
    regions = regionprops(labeled_image)
    if regions:
        region = max(regions, key=lambda r: r.area)
    else:
        return [np.nan] * 8
    return [
        region.area, region.perimeter, region.eccentricity,
        region.solidity, region.extent,
        region.axis_major_length, region.axis_minor_length,
        region.feret_diameter_max
    ]

def process_slices(binary_3d_image, feature_vector_names):
    """Extract shape descriptors for each 2D slice in a 3D stack."""
    feature_vectors = []
    for i in range(binary_3d_image.shape[0]):
        binary_slice = binary_3d_image[i, :, :]
        if np.all(binary_slice == 0):
            feature_vectors.append([np.nan] * len(feature_vector_names))
        else:
            feature_vectors.append(get_shape_descriptors(binary_slice))
    return np.array(feature_vectors)

# --- Feature Extraction ---
feature_vector_names = [
    'area', 'perimeter', 'eccentricity', 'solidity',
    'extent', 'axis_major_length', 'axis_minor_length',
    'feret_diameter_max'
]

merged_df_full_cropped['features_array'] = [[] for _ in range(len(merged_df_full_cropped))]
merged_df_full2 = pd.DataFrame()
bbox_size = 75
total_timepoints = 192

for t in tqdm(range(1, total_timepoints)):
    segmented_image = tifffile.imread(
        f'split_nuclei_membrane_raw/split_cellpose_results_relabelled/final_relabelled_timelapse_fifth_dataset-{t}_cp_masks.tif'
    )
    padded_image = np.pad(segmented_image, ((0, 0), (bbox_size, bbox_size), (bbox_size, bbox_size)), mode='constant', constant_values=0)
    merged_df = merged_df_full_cropped[merged_df_full_cropped['Spot frame'] == (t-1)].copy()
    merged_df['features_array'] = [[] for _ in range(len(merged_df))]
    membrane_labels = merged_df['mem_label'].to_list()
    label_centroid_map = measure_objects(padded_image)
    combined_3d_array = process_label(padded_image, membrane_labels, bbox_size, label_centroid_map)
    feature_vectors = process_slices(combined_3d_array, feature_vector_names)
    for i in range(len(merged_df)):
        merged_df.at[merged_df.index[i], 'features_array'] = feature_vectors[i]
    merged_df_full2 = pd.concat([merged_df_full2, merged_df], ignore_index=True)

# --- Unpack Features and Save ---
unpacked_df = pd.DataFrame(merged_df_full2['features_array'].tolist(), columns=[
    'mem_2d_area', 'mem_2d_perimeter', 'mem_2d_eccentricity', 'mem_2d_solidity',
    'mem_2d_extent', 'mem_2d_axis_major_length', 'mem_2d_axis_minor_length',
    'mem_2d_feret_diameter_max'
])
mem_2d_df = pd.concat([merged_df_full2.drop(columns=['features_array']), unpacked_df], axis=1)
mem_2d_df.to_csv('nuclei_membrane_tracking/membrane_manual_dataset_with2D.csv', index=False)

C:\Users\ghr283\AppData\Local\Temp\ipykernel_4168\82927097.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_full_cropped['features_array'] = [[] for _ in range(len(merged_df_full_cropped))]
100%|██████████| 1/1 [00:02<00:00,  2.51s/it]
